In [1]:
from fastai.learner import *
from fastai.column_data import *
from fastai.structured import *

D:\python\envs\fastai\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH=Path('data/elo')
# C drive path
PATH1 = Path('c:/data/')

In [15]:
# Load train customer
customer = pd.read_csv(PATH/'train.csv')

In [16]:
#get unique customer ids
customer_uniq_id = customer.groupby(['first_active_month','feature_1','feature_2','feature_3'],as_index=True).count().reset_index()
customer_uniq_id = customer_uniq_id.drop(['card_id','target'],axis=1)
customer_uniq_id['cust_index'] = customer_uniq_id.index
# customer_uniq_id.head()
customer_uniq_id.to_feather(f'{PATH1}\customer_uniq_id')

In [17]:
# Load merchants and get unique
merchants =  pd.read_csv(PATH/'merchants.csv')
merchant_id = merchants.drop(['merchant_id','merchant_group_id','numerical_1','numerical_2','category_1','most_recent_sales_range','most_recent_purchases_range','avg_sales_lag3','avg_sales_lag6',
                              'avg_purchases_lag6','active_months_lag6','avg_sales_lag12','avg_purchases_lag12',
                              'active_months_lag12','category_4','category_2','avg_purchases_lag3','active_months_lag3'],axis=1)
merchant_id['merch_index']= merchant_id.index
merchant_id.head()

,merchant_category_id,subsector_id,city_id,state_id,merch_index
0,792,9,242,9,0
1,840,20,22,16,1
2,690,1,-1,5,2
3,792,9,-1,-1,3
4,222,21,-1,-1,4


In [19]:
#Load transactions, drop field not used
transactions= pd.read_csv(PATH/'historical_transactions.csv')
transactions1 = transactions[['card_id','merchant_category_id','subsector_id','state_id','city_id','purchase_amount','purchase_date']].copy()
del(transactions)
# transactions1.head()
# transactions1.to_feather(f'{PATH1}\transactions')

In [20]:
#merge merchants into training set
customer = pd.merge(customer,transactions1, on='card_id', how='left')
customer.to_feather(f'{PATH1}\customer')

In [21]:
#get ids for customers
customer = pd.merge(customer,customer_uniq_id,how='inner',on=['first_active_month','feature_1','feature_2','feature_3'])
customer = customer.drop(['first_active_month','feature_1','feature_2','feature_3'],axis=1)
customer.to_feather(f'{PATH1}\customer')

In [22]:
#reduce size of customers
cust_merch = customer.groupby(['cust_index','merchant_category_id','subsector_id','city_id','state_id'],as_index=False).agg({'purchase_amount' : np.sum}).reset_index().copy()
cust_merch.to_feather(f'{PATH1}\cust_merch')

In [23]:
cust_merch.head()

,index,cust_index,merchant_category_id,subsector_id,city_id,state_id,purchase_amount
0,0,0,19,36,60,16,-2.846532
1,1,0,19,36,158,15,-2.797335
2,2,0,34,38,138,15,-0.598145
3,3,0,38,7,88,16,-0.581616
4,4,0,45,18,138,15,-1.176305


In [24]:
# Load test customer
customer_test = pd.read_csv(PATH/'test.csv')

In [25]:
customer_test = pd.merge(customer_test,customer_uniq_id,how='inner',
                         on=['first_active_month','feature_1','feature_2','feature_3'])
customer_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,cust_index
0,2017-04,C_ID_0ab67a22ab,3,3,1,365
1,2017-04,C_ID_e6340286bd,3,3,1,365
2,2017-04,C_ID_7fa0d03279,3,3,1,365
3,2017-04,C_ID_2fa504fb02,3,3,1,365
4,2017-04,C_ID_967ca675f9,3,3,1,365


In [26]:
customer_test = customer_test.drop(['first_active_month','feature_1','feature_2','feature_3'],axis=1)
customer_test.to_feather(f'{PATH1}\customer_test')

In [28]:
#transfer merchant info to customer_test
len(customer_test)

123603

In [3]:
#load customer tst and merch
customer_test = pd.read_feather(f'{PATH1}\customer_test')
cust_merch = pd.read_feather(f'{PATH1}\cust_merch')

In [4]:
customer_test1 = pd.merge(customer_test,cust_merch,how='left',on=['cust_index'])
customer_test1.to_feather(f'{PATH1}\customer_test1')
customer_test1.head()

MemoryError: 